In [1]:
"""script that take the following data
[14,12,17],
[14,15,12],
[18,17,10],
[19,16,8],
[12,6,5],
[13,15,19],
[11,10,19],
[14,13,21],
[6,5,5],
[7,12,11],
[10,13,6],
[4,9,13],
[4,13,26],
[13,15,9],
[18,16,15],
[21,12,25],
[21,5,13],
[15,5,16],
[7,16,21],
[8,13,14]
which means the processing time of machines M1, M2, M3 for each of the 20 jobs
and calculate the makespan of the schedule that minimizes the makespan
with linear programming using the pulp library
for the Alternative Formulations of a Flow-shop Scheduling
Problem
J. M. WILSON
Department of Industrial Engineering
University of Wisconsin-Madison
Madison, Wisconsin 53706
algorithm"""
import pulp 
import numpy as np
arg = np.array([[14,12,17], [14,15,12], [18,17,10], [19,16,8], [12,6,5], [13,15,19], [11,10,19], [14,13,21], [6,5,5], [7,12,11], [10,13,6], [4,9,13], [4,13,26], [13,15,9], [18,16,15], [21,12,25], [21,5,13], [15,5,16], [7,16,21], [8,13,14]])
J = ["J1","J2","J3","J4","J5","J6","J7","J8","J9","J10","J11","J12","J13","J14","J15","J16","J17","J18","J19","J20"]
M = ["M1","M2","M3"]
# Create the 'prob' variable to contain the problem data
prob = pulp.LpProblem("The Flow-shop Scheduling Problem", pulp.LpMinimize)
# Create problem Variables
# i = job number, i = 1, 2 . . ., N; N is the total number of jobs;
# j = schedule position for processing,] = 1, 2, .. . N; N possible positions;
# k = operation (machine) number k = 1, 2, ... M; M is the total number of machines in the shop;
# Zij 1 if job i is scheduled in positions for processing, 0 otherwise;
Z= pulp.LpVariable.dicts('z',((i,j) for j in J for i in J),cat='Binary')
# Sjk = time at which the job in schedule position j commences processing on machine k.
S= pulp.LpVariable.dicts('s',((j,k) for j in J for k in M),lowBound=0,cat='Continuous')
# add the constraints
# Constraint (1)
for i in J:
    prob += pulp.lpSum(Z[(i,j)] for j in J)==1
# Constraint (2)
for j in J:
    prob += pulp.lpSum(Z[(i,j)] for i in J)==1
# Constraint (3) for j = 1, 2, . . ., N-1  S_{j+1,1}=S_{j,1}+Sum_{i=1}^{N}Z_{ij}p_{ik}
for j in J[:-1]:
    prob += S[(J[J.index(j)+1],M[0])]==S[(j,M[0])]+pulp.lpSum(Z[(i,j)]*arg[J.index(i)][0] for i in J)
# Constraint (4) for S_{1,1}=0
prob += S[(J[0],M[0])]==0
# Constraint (5) for k= 1,2,3,. . .,M-1 S  S_{1,k+1}=S_{1,k}+Sum_{i=1}^{N}Z_{ij}p_{ik}
for k in M[:-1]:
    prob += S[(J[0],M[M.index(k)+1])]==S[(J[0],k)]+pulp.lpSum(Z[(i,J[0])]*arg[J.index(i)][M.index(k)] for i in J)
# Constraint (6) The commencement of job j on machine (k + 1) is later than its finish time on machine k:
# S_{j,k+1}>=S_{j,k}+Sum_{i=1}^{N}Z_{ij}p_{ik} for j = 1, 2, . . ., N; k = 1, 2, . . ., M-1
for j in J:
    for k in M[:-1]:
        prob += S[(j,M[M.index(k)+1])]>=S[(j,k)]+pulp.lpSum(Z[(i,j)]*arg[J.index(i)][M.index(k)] for i in J)
# Constraint (7) The commencement of job (j + 1) on a machine is later than the finish time of a job j on the same machine:
# S_{j+1,k}>=S_{j,k}+Sum_{i=1}^{N}Z_{ij}p_{ik} for j = 1, 2, . . ., N-1; k = 1, 2, . . ., M
for j in J[:-1]:
    for k in M:
        prob += S[(J[J.index(j)+1],k)]>=S[(j,k)]+pulp.lpSum(Z[(i,j)]*arg[J.index(i)][M.index(k)] for i in J)
# Objective Function Minimize the makespan
# Minimize S_{N,M}+sum_{i=1}^{N}sum_{j=1}^{N}Z_{iN}p_{iM}
prob += S[(J[-1],M[-1])]+pulp.lpSum(Z[(i,J[-1])]*arg[J.index(i)][M.index(M[-1])] for i in J)

# The problem data is written to an .lp file
prob.writeLP("FlowShopScheduling.lp")
# The problem is solved using PuLP's choice of Solver
prob.solve()
# The status of the solution is printed to the screen
print("Status:", pulp.LpStatus[prob.status])
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    print(v.name, "=", v.varValue)
# The optimised objective function value is printed to the screen
print("Optimal Solution to the problem: ", pulp.value(prob.objective))
# A list of the jobs in the optimal schedule is created
optimal_schedule = []
for j in J:
    for i in J:
        if Z[(i,j)].varValue == 1:
            optimal_schedule.append(i)
# The optimal schedule is printed to the screen
print("Optimal Schedule: ", optimal_schedule)
# The makespan of the optimal schedule is calculated and printed to the screen
makespan = pulp.value(prob.objective)
print("Makespan of the optimal schedule: ", makespan)




C:\Users\Home\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Status: Optimal
s_('J1',_'M1') = 0.0
s_('J1',_'M2') = 4.0
s_('J1',_'M3') = 13.0
s_('J10',_'M1') = 79.0
s_('J10',_'M2') = 98.0
s_('J10',_'M3') = 118.0
s_('J11',_'M1') = 93.0
s_('J11',_'M2') = 110.0
s_('J11',_'M3') = 135.0
s_('J12',_'M1') = 104.0
s_('J12',_'M2') = 120.0
s_('J12',_'M3') = 154.0
s_('J13',_'M1') = 111.0
s_('J13',_'M2') = 136.0
s_('J13',_'M3') = 175.0
s_('J14',_'M1') = 125.0
s_('J14',_'M2') = 151.0
s_('J14',_'M3') = 187.0
s_('J15',_'M1') = 138.0
s_('J15',_'M2') = 166.0
s_('J15',_'M3') = 206.0
s_('J16',_'M1') = 159.0
s_('J16',_'M2') = 177.0
s_('J16',_'M3') = 219.0
s_('J17',_'M1') = 177.0
s_('J17',_'M2') = 196.0
s_('J17',_'M3') = 229.0
s_('J18',_'M1') = 196.0
s_('J18',_'M2') = 214.0
s_('J18',_'M3') = 237.0
s_('J19',_'M1') = 214.0
s_('J19',_'M2') = 230.0
s_('J19',_'M3') = 252.0
s_('J2',_'M1') = 4.0
s_('J2',_'M2') = 13.0
s_('J2',_'M3') = 26.0
s_('J20',_'M1') = 228.0
s_('J20',_'M2') = 249.0
s_('J20',_'M3') = 273.0
s_('J3',_'M1') = 10.0
s_('J3',_'M2') = 18.0
s_('J3',_'M3') = 31.0
